<a href="https://colab.research.google.com/github/JamilaAferhane/Document_Summarization/blob/main/document_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# !pip install PyPDF2

# Importing necessary modules

In [ ]:
#importation of necessary libraries
import os.path
import nltk
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from gensim.models.coherencemodel import CoherenceModel
import PyPDF2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Converting pdf to text

In [ ]:
def convert_pdf_to_txt(pdf_path, txt_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text_content = ''

        # Find the start and end pages for each section
        abstract_start_page = None
        intro_start_page = None
        conclusion_start_page = None
        references_start_page = None

        for i, page in enumerate(pdf_reader.pages):
            page_text = page.extract_text()

            if not abstract_start_page and ('Abstract' in page_text or 'ABSTRACT' in page_text ):
                abstract_start_page = i
                print ('I', i)

            if not intro_start_page and ('Introduction' in page_text or 'INTRODUCTION' in page_text):
                intro_start_page = i
                print('II', i)

            if not conclusion_start_page and ('Conclusion' in page_text or 'CONCLUSION' in page_text ):
                conclusion_start_page = i
                print('III', i)

            if not references_start_page and ('References' in page_text or 'REFERENCES' in page_text or 'Bibliography' in page_text or  'BIBLIOGRAPHY' in page_text ):
                references_start_page = i
                print('IIII', i)

        # Determine the start and end pages for each section
        start_page = None
        end_page = None

        if intro_start_page is not None:
            start_page = intro_start_page
            print('a')

        elif abstract_start_page is not None:
            start_page = abstract_start_page
            print('b')



        if conclusion_start_page is not None:
            end_page = conclusion_start_page
            print('c')
            print('end page', end_page)

        elif references_start_page is not None:
            end_page = references_start_page
            print('d')

        # Extract the text based on the determined page range
        for i, page in enumerate(pdf_reader.pages):

            if (start_page is not None) and (i >= start_page) and (end_page is None or i < end_page):
                print('Entering the considered part of the pdf')

                if i == start_page : #and start_page != 0  :


                    page_text = page.extract_text()

                    if ('INTRODUCTION' in page_text):
                        text_content += page_text.split("INTRODUCTION",1)[1]
                        print('Everthing before \'Introduction\' is not included')

                    elif ('Introduction' in page_text):
                        text_content += page_text.split("Introduction",1)[1]
                        print('Everthing before \'Introduction\': is not included')

                    elif ('Abstract' in page_text):
                        text_content += page_text.split("Abstract",1)[1]
                        print('Everthing before \'Abstract\' is not included')

                    elif ('ABSTRACT' in page_text):
                        text_content += page_text.split("ABSTRACT",1)[1]
                        print('Everthing before \'Abstract\': is not included')


                else:
                    text_content += page.extract_text()
                    print('A whole page is added')

            elif ( start_page is None and (end_page is None or i<end_page)):
                text_content += page.extract_text()

            if i == end_page:

                page_text = page.extract_text()

                if any(keyword in page_text for keyword in ('Conclusion', 'CONCLUSION')):
                    print ("there is a conclusion")
                    conclusion_start_index = page_text.index(keyword)
                    # Find the index of the next title after the conclusion
                    next_title_index = -1
                    for title in ['Acknowledgements','ACKNOWLEDGEMENTS','Acknowledgement','ACKNOWLEDGEMENT', 'Acknowledgments'
                                  ,'ACKNOWLEDGMENTS','Acknowledgment','ACKNOWLEDGMENT', 'Appendix', 'APPENDIX','References',
                                  'REFERENCES', 'Bibliography', 'BIBLIOGRPHY', 'Works Cited', 'WORKS CITED', 'Appendices',
                                  'APPENDICES', 'Glossary', 'GLOSSARY','Index', 'List of Figures', 'List of Tables',
                                  'About the Author(s)', 'Notes', 'INDEX', 'LIST OF FIGURES', 'LIST OF TABLES',
                                  'ABOUT THE AUTHOR(S)', 'NOTES', 'About the author', 'ABOUT THE AUTHOR',
                                  'acknowledgements', 'acknowledgements', 'acknowledgement', 'acknowledgement',
                                  'acknowledgments','acknowledgments', 'acknowledgment', 'acknowledgment', 'appendix',
                                  'appendix', 'references','references', 'bibliography', 'bibliography', 'works cited',
                                  'works cited', 'appendices', 'appendices', 'glossary', 'glossary', 'index',
                                  'list of figures', 'list of tables', 'about the author(s)', 'notes', 'index',
                                  'list of figures', 'list of tables', 'about the author(s)', 'notes', 'about the author']:
                        #conclusion_start_index: The starting index from which the search should begin within
                        title_index = page_text.find(title, conclusion_start_index)
                        print(title, title_index)
                        if title_index != -1 and (next_title_index == -1 or title_index < next_title_index):
                            print(title)
                            next_title_index = title_index
                    if conclusion_start_index != -1:
                        conclusion_content = page_text[conclusion_start_index:next_title_index].strip()
                        text_content += conclusion_content
                    else:
                        text_content += page_text[conclusion_start_index:]

                elif ('REFERENCES' in page_text):
                    text_content += page_text.split("References",1)[0]
                    print('There is a references part but it is not considered')

                elif ('References' in page_text):
                    text_content += page_text.split("References",1)[0]
                    print('There is a references part but it is not considered')

                elif ('Bibliography' in page_text or 'BIBLIOGRAPHY'):
                    text_content += page_text.split("Bibliography",1)[0]
                    print('There is a bibliography part but it is not considered')
                elif ('BIBLIOGRAPHY' in page_text ):
                    text_content += page_text.split("Bibliography",1)[0]
                    print('There is a bibliography part but it is not considered')

        cleaned_text = text_content.strip()  # Remove leading/trailing whitespaces
        cleaned_text = cleaned_text.replace('\n', ' ')  # Replace all line breaks with spaces
        cleaned_text = cleaned_text.replace('. ', '.\n')  # Add line breaks after periods

        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(cleaned_text)


# Usage example
# pdf_path = './pdf_for_test/test_7.pdf'
# txt_path = './generated_texts/test_s.txt'
# convert_pdf_to_txt(pdf_path, txt_path)


# def convert_pdf_to_txt(pdf_path, txt_path):
#     with open(pdf_path, 'rb') as pdf_file:
#         pdf_reader = PyPDF2.PdfReader(pdf_file)
#         text_content = ''

#         for page in pdf_reader.pages:
#             text_content += page.extract_text()

#         with open(txt_path, 'w', encoding='utf-8') as txt_file:
#             txt_file.write(text_content)

# Loading the document

In [ ]:
 # Load the document
def convert_to_sentences(text):
    """
    Input  : text document
    Purpose: convert text into a list of sentences
    Output : list of sentences
    """
    sentences = sent_tokenize(text)
    return sentences

def load_data(path, file_name):
    """
    Input  : path and file_name
    Purpose: loading text file
    Output : list of paragraphs/documents and
             title (initial 100 characters considered as the title of each document)
    """
    documents_list = []
    with open(os.path.join(path, file_name), "r") as fin:
        text = fin.read()
        sentences = convert_to_sentences(text)
        # Split the document into sentences
        for sentence in sentences:
            # Append the current sentence to the current document
            documents_list.append(sentence)
            # Extract the first 100 characters (or less) as the title
    print("Total Number of Sentences:", len(documents_list))
    return documents_list





#Example usage

# pdf_path = '/content/drive/My Drive/test.pdf'
# txt_path = '/content/drive/My Drive/test.txt'
# convert_pdf_to_txt(pdf_path, txt_path)
# document_list = load_data("","/content/drive/My Drive/test.txt")
# print(document_list)



I 1
II 1
IIII 10
a
d
Entering the considered part of the pdf
Everthing before 'Introduction': is not included
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
There is a references part but it is not considered
Total Number of Sentences: 86
['Latent Semantic Analysis (LSA) is a technique for creating vector-based representations of texts which are claimed to capture their semantic con- tent.', 'The primary function of LSA is to compute the similarity of text pairs 1by comparing their vector representations.', 'This relatively simple similarity met

# Preprocessing the document

In [ ]:
# Preprocess the document
def preprocess_data(doc_set):
    """Input  : document list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # add tokens to list
        texts.append(stemmed_tokens)
    return texts

# Extracting keywords

In [ ]:
def extract_keywords(text, num_keywords=5):

    # Flatten the list of lists into a single list of tokens
    tokens = [token for sublist in text for token in sublist]

    tf_vectorizer = CountVectorizer()
    tfidf_transformer = TfidfTransformer()

    tf_matrix = tf_vectorizer.fit_transform([' '.join(tokens)])
    tfidf_matrix = tfidf_transformer.fit_transform(tf_matrix)

    feature_names = tf_vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.toarray()[0]

    keywords = [feature_names[idx] for idx in tfidf_scores.argsort()[-num_keywords:][::-1]]

    return keywords


# Preparing the term frequency matrix of the document

In [ ]:
def prepare_corpus(doc_clean):
    """
    Input  : clean document
    Purpose: create term dictionary of our courpus and Converting list of documents (corpus) into Document Term Matrix
    Output : term dictionary and Document Term Matrix
    """
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model
    return dictionary,doc_term_matrix

# Create a gensim latent semantic

In [ ]:
def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel

# Select the number of topics the documents have to be divided into

In [ ]:
number_of_topics=2
words=20

pdf_path = '/content/drive/My Drive/test.pdf'
txt_path = '/content/drive/My Drive/test.txt'
convert_pdf_to_txt(pdf_path, txt_path)
document_list=load_data("","/content/drive/My Drive/test.txt")
clean_text=preprocess_data(document_list)
keywords = extract_keywords(clean_text)
dict1,doc_term_matrix=prepare_corpus(clean_text)
model=create_gensim_lsa_model(clean_text,number_of_topics,words)
corpus_lsi = model[doc_term_matrix]

I 1
II 1
IIII 10
a
d
Entering the considered part of the pdf
Everthing before 'Introduction': is not included
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
Entering the considered part of the pdf
A whole page is added
There is a references part but it is not considered
Total Number of Sentences: 86
[(0, '0.473*"lsa" + 0.406*"word" + 0.255*"document" + 0.249*"see" + 0.151*"vector" + 0.135*"2005" + 0.132*"corpu" + 0.122*"use" + 0.119*"represent" + 0.106*"applic" + 0.105*"student" + 0.103*"text" + 0.097*"similar" + 0.094*"help" + 0.086*"base" + 0.086*"grade" + 0.086*"summari" + 0.086*"evalu" + 0.086*"essay" 

In [ ]:
for doc, as_text in zip(corpus_lsi, document_list):
    print(doc, as_text)

[(0, 1.153840519596598), (1, -0.5633294910394763)] Latent Semantic Analysis (LSA) is a technique for creating vector-based representations of texts which are claimed to capture their semantic con- tent.
[(0, 1.1046627524030228), (1, -0.43781962012493486)] The primary function of LSA is to compute the similarity of text pairs 1by comparing their vector representations.
[(0, 0.7560582240652032), (1, -0.2889067628937494)] This relatively simple similarity metric has been situated within a psychological theory of text meaning and has been shown to closely match human capabilities on a variety of tasks.
[(0, 1.8813663627116441), (1, -0.3990646271206591)] This article follows the developmental path of LSA, describing its histor- ical context, showing how it computes and uses its vector representations, and then giving examples of the theoretical and empirical support for LSA and its current research directions.
[(0, 2.0375292895239103), (1, -0.37372896615587753)] 2 How LSA works LSA (origina

In [ ]:
def takenext(elem):
	"""
	sort
	"""
	return elem[1]

# Sort the vectors in descending order of weightage

In [ ]:
#sort each vector by score
vecsSort = list(map(lambda i: list(), range(2))) #output: [[ ], [ ]]
for i,docv in enumerate(corpus_lsi):
	for sc in docv:
		isent = (i, abs(sc[1]))
		vecsSort[sc[0]].append(isent)
vecsSort = list(map(lambda x: sorted(x,key=takenext,reverse=True), vecsSort))

In [ ]:
print(vecsSort)

[[(55, 5.902721989878568), (47, 3.79724270375292), (72, 2.716209441682509), (82, 2.2081071892056254), (63, 2.1915563917111336), (64, 2.120854648201684), (75, 2.043800876806583), (4, 2.0375292895239103), (3, 1.8813663627116441), (73, 1.7343874898780196), (66, 1.6633724125061045), (5, 1.5814615923125204), (54, 1.5724491971808008), (37, 1.4910660614278182), (21, 1.437305966873764), (71, 1.4144712581832803), (52, 1.3934374450445515), (56, 1.3557480856971749), (12, 1.3508473230275055), (69, 1.3227136028212618), (70, 1.2969885943285426), (46, 1.296104340686351), (33, 1.2417798964632636), (43, 1.2032988406040208), (0, 1.153840519596598), (48, 1.1400378759912824), (1, 1.1046627524030228), (29, 1.0722060930936534), (49, 1.033969481853749), (42, 1.0311700781177466), (6, 1.0024028267550074), (41, 0.9451572292619284), (11, 0.8946941013589998), (81, 0.8801484544865408), (22, 0.8657510155570832), (85, 0.8533913731553561), (50, 0.801305604470058), (79, 0.7830454311125645), (84, 0.764667690405409), (2

# Select the sentences for the summary

In [ ]:
sentIndexes = set()

In [ ]:
def selectTopSent(summSize, numTopics, sortedVec):
  topSentences = []
  sent_no = []
  sentInd = set()
  sCount = 0
  for i in range(summSize):
    for j in range(numTopics):
      vecs = sortedVec[j]
      si = vecs[i][0]
      if si not in sentInd:
        sent_no.append(si)
        topSentences.append(vecs[i])
        sentInd.add(si)
        sCount +=1
        if sCount == summSize:
          return sent_no

In [ ]:
topSentences = selectTopSent(8, 2, vecsSort)

In [ ]:
print(topSentences)
topSentences.sort()
print(topSentences)

[55, 47, 72, 82, 63, 64, 12, 75]
[12, 47, 55, 63, 64, 72, 75, 82]


In [ ]:
summary = []
doc = []
cnt = 0
for sentence in document_list:
  doc.append(sentence)
  if cnt in topSentences:
    summary.append(sentence)
  cnt += 1
summary = " ".join(summary)
doc = " ".join(doc)
print("\n")
print("Original:")
print(doc)
print("Summary:")
print(summary)
print("Keywords:")
print(keywords)



Original:
Latent Semantic Analysis (LSA) is a technique for creating vector-based representations of texts which are claimed to capture their semantic con- tent. The primary function of LSA is to compute the similarity of text pairs 1by comparing their vector representations. This relatively simple similarity metric has been situated within a psychological theory of text meaning and has been shown to closely match human capabilities on a variety of tasks. This article follows the developmental path of LSA, describing its histor- ical context, showing how it computes and uses its vector representations, and then giving examples of the theoretical and empirical support for LSA and its current research directions. 2 How LSA works LSA (originally known as Latent Semantic Indexing) was developed for the task of Information Retrieval, that is, selecting from a large database of documents a few relevant documents which match a given query . Previ- ous approaches to this task included keywor